<a href="https://colab.research.google.com/github/AyoubMDL/doctorFalcon/blob/main/doctorFalcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [ ]:
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 82.1 

In [ ]:
!pip install -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers (to revision de9255de27abfcae4a1f816b904915f0b1e23cd9) to /tmp/pip-req-build-qshu5nhb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qshu5nhb
  Running command git rev-parse -q --verify 'sha^de9255de27abfcae4a1f816b904915f0b1e23cd9'
  Running command git fetch -q https://github.com/huggingface/transformers de9255de27abfcae4a1f816b904915f0b1e23cd9
  Running command git checkout -q de9255de27abfcae4a1f816b904915f0b1e23cd9
  Resolved https://github.com/huggingface/transformers to commit de9255de27abfcae4a1f816b904915f0b1e23cd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.30.0.dev0-py3-none-any.whl size=7128533 sha256=56337d

## Loading the dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
                    'json',
                    data_files={
                        "train": "/content/drive/MyDrive/work/medical_chat_data.json"
                    }
                )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-c807fbf69db0ccb3/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import re

def construct_text_field(example):
    pattern = r'\[\|(.+?)\|\] (.+)'
    matches = re.findall(pattern, example['input'])

    human = []
    ai = []
    text_field = ""

    for speaker, message in matches:
        if speaker.lower() == "human":
            text_field += "### Human: "
            text_field += message
            text_field += " "
        else:
            text_field += "###Assistant: "
            text_field += message
            text_field += " "

    example['text'] = text_field

    return example

In [ ]:
dataset = dataset.map(construct_text_field)

Map:   0%|          | 0/46867 [00:00<?, ? examples/s]

In [ ]:
dataset['train']['text'][45478]

"### Human: Can you tell me if there are any safety concerns or special precautions about Omacetaxine Injection? ###Assistant: Yes, there are some safety concerns and precautions associated with Omacetaxine Injection. It is important to discuss your medical history and any allergies you may have with your healthcare provider before receiving this injection. ### Human: What are some common side effects of Omacetaxine Injection? ###Assistant: Some common side effects include nausea, vomiting, fatigue, headache, and anemia. However, it is important to note that everyone's experience may be different and some people may not experience any side effects at all. ### Human: Are there any medications that interact with Omacetaxine Injection? ###Assistant: Yes, some medications may interact with Omacetaxine Injection, such as blood thinners and medications that affect the liver. It is important to talk to your healthcare provider about all your medications before receiving this injection. ### Hu

## Loading the model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
num_train_epochs = 1
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/46867 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:293: UserWarning: Found 0 samples with a length of 512. You might want to decrease the `max_seq_len` argument.
  warnings.warn(


In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.094400
20,0.008000
30,0.001400
40,0.000700
50,0.000600
60,0.000600
70,0.000600
80,0.000600
90,0.000600
100,0.000600


TrainOutput(global_step=500, training_loss=0.002590940507128835, metrics={'train_runtime': 4778.3209, 'train_samples_per_second': 1.674, 'train_steps_per_second': 0.105, 'total_flos': 1.5869138632704e+16, 'train_loss': 0.002590940507128835, 'epoch': 500.0})

In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
trainer.model.push_to_hub("AyoubMDL/private_doctor", private=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/522M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AyoubMDL/private_doctor/commit/fd12900ed46372ff8b00dc45755d78228e1f1d79', commit_message='Upload model', commit_description='', oid='fd12900ed46372ff8b00dc45755d78228e1f1d79', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.tokenizer.push_to_hub("AyoubMDL/private_doctor", private=True)

CommitInfo(commit_url='https://huggingface.co/AyoubMDL/private_doctor/commit/232760877558e174af59bcf8685ae26a6a036a0b', commit_message='Upload tokenizer', commit_description='', oid='232760877558e174af59bcf8685ae26a6a036a0b', pr_url=None, pr_revision=None, pr_num=None)

## Loading the trained model from huggging face

In [ ]:
from peft import get_peft_model
import torch
import transformers
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from transformers import AutoTokenizer

lora_config = LoraConfig.from_pretrained("AyoubMDL/private_doctor")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained("AyoubMDL/private_doctor")
model = AutoModelForCausalLM.from_pretrained(
    lora_config.base_model_name_or_path,
    quantization_config=bnb_config,
    device_map={"":0})

Loading tiiuae/falcon-7b-instruct requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b-instruct. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading tiiuae/falcon-7b-instruct requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b-instruct. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
from torch import cuda, bfloat16

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device

'cuda:0'

## Stopping criteria

In [ ]:
tokenizer.eos_token, tokenizer.eos_token_id

('<|endoftext|>', 11)

In [ ]:
tokenizer.convert_tokens_to_ids(['User', ':'])

[7932, 37]

In [ ]:
tokenizer.convert_tokens_to_ids(['###','Human', ':'])


[19468, 23431, 37]

In [ ]:
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [
       ['###','Human', ':'], ['User', ':']
    ]
]
stop_token_ids

[[19468, 23431, 37], [7932, 37]]

In [ ]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([19468, 23431,    37], device='cuda:0'),
 tensor([7932,   37], device='cuda:0')]

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
stopping_criteria(
    torch.LongTensor([[1, 2, 19468, 23431, 37]]).to(device),
    torch.FloatTensor([0.0])
)

True

## Building the pipeline

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    device=device,
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model will ramble
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=256,  # max number of tokens to generate in the output
    repetition_penalty=1.2  # without this output begins repeating
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instruction with no input
prompt = PromptTemplate(
    input_variables=["query"],
    template="""You are a helpful AI assistant in the medical field, you will answer the users query
with a short but precise answer. If you are not sure about the answer you state
"I don't know". This is a conversation, not a webpage, there should be ZERO HTML
in the response.

Remember, Assistant responses are short. Here is the conversation:

### Human: {query}
### Assistant: """
)

llm = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
output = llm_chain.predict(
    query="Can you tell me what are the HIV symptoms?"
).lstrip()
print(output)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


The HIV symptoms can vary from person to person, but some common ones include fever, fatigue, night sweats, weight loss, and swollen lymph nodes. It's important to note that not everyone with HIV will experience symptoms, and it can take years for symptoms to develop. It's also possible to have HIV without experiencing any symptoms at all.

Is there anything else you would like to know about HIV?
User 


In [ ]:
print(output.removesuffix('User '))

The HIV symptoms can vary from person to person, but some common ones include fever, fatigue, night sweats, weight loss, and swollen lymph nodes. It's important to note that not everyone with HIV will experience symptoms, and it can take years for symptoms to develop. It's also possible to have HIV without experiencing any symptoms at all.

Is there anything else you would like to know about HIV?



In [ ]:
output = llm_chain.predict(
    query="Can you tell me how to stop blood ?"
).lstrip()
print(output)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


To stop blood, you can try applying pressure to the affected area. You can also try elevating the area if possible. If the bleeding is severe, seek medical attention immediately.

Is there anything else I can help you with?
User 


In [ ]:
output = llm_chain.predict(
    query="I have stomacache everytime I drink milk, what is the cause of it? and give me some solutions."
).lstrip()
print(output)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Stomachache after drinking milk can be caused by lactose intolerance. To avoid this, try consuming smaller amounts of milk or try lactose-free alternatives. Additionally, you can try eating more slowly and avoiding foods that are high in fat.

If the problem persists, it is recommended to consult a doctor for further evaluation.


In [ ]:
output = llm_chain.predict(
    query="I have stomacache everytime I drink milk, what is the cause of it? and give me some solutions with medication."
).lstrip()
print(output)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Stomachache after drinking milk can be caused by lactose intolerance. You can try taking lactase enzyme supplements or switching to non-dairy milk alternatives. Additionally, avoiding dairy products for a few hours after consuming milk can help alleviate symptoms.

I hope this helps!


In [ ]:
output = llm_chain.predict(
    query="Everytime I am exposed to sunlight, I start feeling weak and tired. I breath hardly. What are the causes of it? and give me some solutions with medication."
).lstrip()
print(output)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


There could be several reasons for your symptoms. One possibility is that you may be experiencing sun poisoning, which can be caused by overexposure to sunlight. This can result in dehydration, heat exhaustion, and other symptoms. To treat this, you should try to stay hydrated and avoid prolonged exposure to the sun. Additionally, you may want to consider taking a break from sunbathing and seeking shade when possible.

If you are experiencing these symptoms, it is important to seek medical attention. Your doctor can help determine the cause of your symptoms and recommend appropriate treatment. In the meantime, you can try to stay hydrated and avoid prolonged exposure to the sun.

I hope this information helps. Do you have any other questions?
User 


In [ ]:
from typing import Any, Dict, List, Optional

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate

class DoctorFalconChain(Chain):
    prompt: BasePromptTemplate
    llm: BaseLanguageModel
    output_key: str = "text"
    suffixes = ['', 'User:', 'User ', 'system:', 'Assistant:']

    @property
    def input_keys(self) -> List[str]:
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # format the prompt
        prompt_value = self.prompt.format_prompt(**inputs)
        # generate response from llm
        response = self.llm.generate_prompt(
            [prompt_value],
            callbacks=run_manager.get_child() if run_manager else None
        )
        # _______________
        # here we add the removesuffix logic
        for suffix in self.suffixes:
            response.generations[0][0].text = response.generations[0][0].text.removesuffix(suffix)

        return {self.output_key: response.generations[0][0].text.lstrip()}

    async def _acall(
        self, inputs: Dict[str, Any], run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        raise NotImplementedError("Async is not supported for this chain.")

    @property
    def _chain_type(self) -> str:
        return "open_llama_chat_chain"

    def predict(self, query: str) -> str:
        out = self._call(inputs={'query': query})
        return out['text']

In [ ]:
llama_chain = DoctorFalconChain(llm=llm, prompt=prompt)

In [ ]:
output = llama_chain.predict(
    query="Everytime I am exposed to sunlight, I start feeling weak and tired. I breath hardly. What are the causes of it? and give me some solutions with medication."
)
print(output)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


There could be several reasons for your symptoms. One possibility is that you may be experiencing sun poisoning, which can be caused by overexposure to sunlight. This can result in dehydration, heat exhaustion, and other symptoms. To treat this, you should try to stay hydrated and avoid prolonged exposure to the sun. Additionally, you may want to consider taking a break from sunbathing and seeking shade when possible.

If you are experiencing these symptoms, it is important to seek medical attention. Your doctor can help determine the cause of your symptoms and recommend appropriate treatment. In the meantime, you can try to stay hydrated and avoid prolonged exposure to the sun.

I hope this information helps. Do you have any other questions?

